# 1° Passo: Import das bibliotecas necessárias

In [4]:
import os, shlex, subprocess
from ultralytics import YOLO
import glob
import tkinter as tk
from tkinter import filedialog

# 2° Passo: Coletar o arquivo que será utilizado

Para isso foi feita uma janela simples para coletar o caminho do arquivo que será utilizado para a detecção

In [62]:

root = tk.Tk()
root.withdraw()  # não mostrar a janela principal

input_path = filedialog.askopenfilename(
    title="Selecione um arquivo",
    filetypes=[("Vídeos", "*.mp4 *.avi *.mkv"), ("Todos os arquivos", "*.*")]
)

if not input_path:
    raise RuntimeError("Nenhum arquivo selecionado")

print("Arquivo selecionado:", input_path)

Arquivo selecionado: /home/gabriel/Downloads/video_trabalho.mp4


# 3° Passo: Transcodificação

Após a coleta dos dados, fazemos uma transcodificação utilizando o FFMPEG

In [63]:
# H.264 e AAC funcionam muito bem com YOLO
nome_arq = os.path.splitext(os.path.basename(input_path))[0]
video_h264 = f"{nome_arq}_h264.mp4"

# Só transcodifica se o arquivo transcodado não existir ainda
if not os.path.exists(video_h264):
    cmd = f'ffmpeg -y -i {shlex.quote(input_path)} -c:v libx264 -preset medium -crf 18 -pix_fmt yuv420p -c:a aac -b:a 192k {shlex.quote(video_h264)}'
    print("Executando ffmpeg...")
    subprocess.run(cmd, shell=True, check=True)
    print("Transcodificação concluída:", video_h264)
else:
    print("Arquivo transcodado já existe:", video_h264)

Arquivo transcodado já existe: video_trabalho_h264.mp4


# 4° Passo: Inferência do modelo

Utilizamos o modelo yolov9c para realizar a inferência

Parâmetros:
- source: caminho do vídeo
- imgsz: resolução de inferência (ex.: 640)
- conf: confidence threshold (ex.: 0.25)
- save: salva imagens/video resultante
- save_txt: salva arquivos txt com boxes
- device: 'cpu' ou '0' para GPU (Colab GPU: normalmente '0')

In [64]:
model = YOLO('yolov9c.pt')

print("Iniciando inferência...")
results = model.predict(source=video_h264, imgsz=640, conf=0.25, save=True, save_txt=False, device='0')
print("Inferência concluída. Resultado salvo em runs/detect/... (pasta da execução)")

Iniciando inferência...

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/783) /home/gabriel/Códigos_VSC/Python/Sistemas_Multimidia/Sistemas_Multimidias/video_trabalho_h264.mp4: 640x384 1 person, 1 dog, 17.4ms
video 1/1 (frame 2/783) /home/gabriel/Códigos_VSC/Python/Sistemas_Multimidia/Sistemas_Multimidias/video_trabalho_h264.mp4: 640x384 1 person, 1 dog, 14.2ms
video 1/1 (frame 3/783) /home/gabriel/Códigos_VSC/Python/Sistemas_Multimidia/Sistemas_Multimidias/video_trabalho_h264.mp4:

# 5° Passo: Coletar áudio

Coletar o áudio do vídeo transcodificado h264

In [65]:
candidates = sorted(glob.glob('runs/detect/**/*', recursive=True), key=os.path.getmtime)
candidates = [c for c in candidates if os.path.splitext(c)[1].lower() in ('.mp4', '.mov', '.mkv', '.avi')]
if not candidates:
    raise RuntimeError("Não encontrei vídeo em runs/detect/. Verifique se o step de inferência salvou o vídeo.")
processed_video = candidates[-1]
print("Vídeo processado encontrado:", processed_video)

# garante que exista o vídeo original de entrada
try:
    src_video = video_h264
except NameError:
    src_video = input_path
print("Usarei este vídeo como fonte do áudio:", src_video)

Vídeo processado encontrado: runs/detect/predict2/video_trabalho_h264.avi
Usarei este vídeo como fonte do áudio: video_trabalho_h264.mp4


In [ ]:
src = processed_video
src = src_video 
out_audio = "original_audio.aac"

if not os.path.exists(out_audio):
    cmd_copy = f'ffmpeg -y -i {shlex.quote(src)} -vn -acodec copy {shlex.quote(out_audio)}'
    r = subprocess.run(cmd_copy, shell=True)
    if r.returncode != 0:

        cmd_enc = f'ffmpeg -y -i {shlex.quote(src)} -vn -c:a aac -b:a 192k {shlex.quote(out_audio)}'
        subprocess.run(cmd_enc, shell=True, check=True)
print("Áudio salvo em:", out_audio)

Áudio salvo em: original_audio.aac


# 6° Passo: Juntando vídeo e áudio

Executando comando em FFMPEG para juntar os arquivos de vídeo e áudio

In [ ]:
proc_vid = processed_video  
audio = "original_audio.aac"
out_final = "Video_final.mp4"


cmd = f'ffmpeg -y -i {shlex.quote(proc_vid)} -i {shlex.quote(audio)} -c:v copy -c:a aac -b:a 192k -map 0:v:0 -map 1:a:0 -shortest {shlex.quote(out_final)}'
print("Executando:", cmd)
subprocess.run(cmd, shell=True, check=True)
print("Arquivo final com áudio gerado em:", out_final)


Executando: ffmpeg -y -i runs/detect/predict2/video_trabalho_h264.avi -i original_audio.aac -c:v copy -c:a aac -b:a 192k -map 0:v:0 -map 1:a:0 -shortest Video_final.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Arquivo final com áudio gerado em: Video_final.mp4


frame=  783 fps=0.0 q=-1.0 Lsize=   34367kB time=00:00:32.60 bitrate=8635.8kbits/s speed= 102x    
video:33574kB audio:773kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.057575%
[aac @ 0x5872a849dec0] Qavg: 270.929


# 7° Passo: Executar o vídeo

Função para executar o vídeo em qualquer sistema operacional

In [69]:
import os
import platform
import subprocess

def abrir_video(video_path):
    sistema = platform.system()

    if sistema == "Windows":
        os.startfile(video_path)
    elif sistema == "Darwin":  # macOS
        subprocess.run(["open", video_path])
    elif sistema == "Linux":
        try:
            subprocess.run(["xdg-open", video_path])  # abre no player padrão
        except FileNotFoundError:
            print("xdg-open não encontrado. Tente instalar VLC ou MPV e rodar diretamente.")
    else:
        print(f"Sistema {sistema} não suportado automaticamente.")


video_path = "Video_final.mp4"
abrir_video(video_path)

[000056fecb06da00] main libvlc: Executando o VLC com a interface padrão. Use 'cvlc' para usar o VLC sem interface.
